# Limited Amplitude - Example Workflow

This notebook demonstrates the complete workflow for applying distance truncation to acoustic bird detection data.

In [ ]:
from limited_amplitude import occurrence, distance_truncation
import pandas as pd

## 1. Define File Paths

Update these paths to point to your data files.

In [ ]:
# WildTrax exports
tags_path = "path/to/your/tags_report.csv"
recordings_path = "path/to/your/recording_report.csv"

# Metadata file (location, canopy, SM2 columns)
metadata_path = "path/to/your/metadata.csv"

## 2. Create Occurrence Matrix (Without Distance Truncation)

This creates a simple occurrence matrix (1/0) from your tags data.

In [ ]:
# Create occurrence dataframe (no distance truncation)
df_occ = occurrence.create_occurrence_dataframe(
    tags_csv_path=tags_path,
    recordings_csv_path=recordings_path,
    filter_vocalization='Song'
)

print(f"Occurrence matrix shape: {df_occ.shape}")
df_occ.head()

## 3. Load Metadata

Load location metadata including ARU type (SM2/SM4) and canopy openness.

In [ ]:
metadata_df = distance_truncation.load_metadata(metadata_file=metadata_path)
print(f"Loaded metadata for {len(metadata_df)} locations")

## 4. Apply Distance Truncation

Filter individual tags based on amplitude thresholds for a specific distance.

In [ ]:
# Apply distance truncation to individual tags
truncated_tags = distance_truncation.apply_distance_truncation(
    tags_csv_path=tags_path,
    metadata_df=metadata_df,
    distance_threshold=150.0  # Distance in meters
)

print(f"Retained {len(truncated_tags)} tags within 150m")

## 5. Convert to Occurrence Format

Convert truncated tags to occurrence matrix (1/0).

In [ ]:
occurrence_df = distance_truncation.convert_to_occurrence(
    truncated_tags=truncated_tags,
    recordings_csv_path=recordings_path
)

print(f"Occurrence matrix shape: {occurrence_df.shape}")
occurrence_df.head()

## 6. Convert to Count Format

Convert truncated tags to count matrix (number of detections per species).

In [ ]:
count_df = distance_truncation.convert_to_counts(
    truncated_tags=truncated_tags,
    recordings_csv_path=recordings_path
)

print(f"Count matrix shape: {count_df.shape}")
count_df.head()

## 7. Save Results

In [ ]:
# Save to CSV
occurrence_df.to_csv('occurrence_150m.csv', index=False)
count_df.to_csv('count_150m.csv', index=False)

print("Results saved!")

## 8. Compare Multiple Distance Thresholds

You can apply different distance thresholds and compare the results.

In [ ]:
# Try different thresholds
thresholds = [50, 100, 150, 200]

for threshold in thresholds:
    truncated = distance_truncation.apply_distance_truncation(
        tags_csv_path=tags_path,
        metadata_df=metadata_df,
        distance_threshold=threshold
    )
    
    occ = distance_truncation.convert_to_occurrence(
        truncated_tags=truncated,
        recordings_csv_path=recordings_path
    )
    
    # Save with threshold in filename
    occ.to_csv(f'occurrence_{threshold}m.csv', index=False)
    print(f"{threshold}m: {len(truncated)} tags retained")